In [1]:
import pandas as pd
import requests
from pprint import pprint
import plotly_express as px
from fouronetwo import Fetch
fetch = Fetch()
import re
from tqdm import tqdm

In [2]:
drive_summary = fetch.drive_analysis(year = 2024, agg_method='weekly')

https://site.api.espn.com/apis/site/v2/sports/football/nfl/scoreboard?limit=1000&dates=2024&seasontype=2&week=1
https://site.api.espn.com/apis/site/v2/sports/football/nfl/scoreboard?limit=1000&dates=2024&seasontype=2&week=2
https://site.api.espn.com/apis/site/v2/sports/football/nfl/scoreboard?limit=1000&dates=2024&seasontype=2&week=3
https://site.api.espn.com/apis/site/v2/sports/football/nfl/scoreboard?limit=1000&dates=2024&seasontype=2&week=4
https://site.api.espn.com/apis/site/v2/sports/football/nfl/scoreboard?limit=1000&dates=2024&seasontype=2&week=5
https://site.api.espn.com/apis/site/v2/sports/football/nfl/scoreboard?limit=1000&dates=2024&seasontype=2&week=6
https://site.api.espn.com/apis/site/v2/sports/football/nfl/scoreboard?limit=1000&dates=2024&seasontype=2&week=7
https://site.api.espn.com/apis/site/v2/sports/football/nfl/scoreboard?limit=1000&dates=2024&seasontype=2&week=8
https://site.api.espn.com/apis/site/v2/sports/football/nfl/scoreboard?limit=1000&dates=2024&seasontype=2

100%|██████████| 166/166 [00:48<00:00,  3.45it/s]


https://site.api.espn.com/apis/site/v2/sports/football/nfl/scoreboard?limit=1000&dates=2024&seasontype=2&week=1
https://site.api.espn.com/apis/site/v2/sports/football/nfl/scoreboard?limit=1000&dates=2024&seasontype=2&week=2
https://site.api.espn.com/apis/site/v2/sports/football/nfl/scoreboard?limit=1000&dates=2024&seasontype=2&week=3
https://site.api.espn.com/apis/site/v2/sports/football/nfl/scoreboard?limit=1000&dates=2024&seasontype=2&week=4
https://site.api.espn.com/apis/site/v2/sports/football/nfl/scoreboard?limit=1000&dates=2024&seasontype=2&week=5
https://site.api.espn.com/apis/site/v2/sports/football/nfl/scoreboard?limit=1000&dates=2024&seasontype=2&week=6
https://site.api.espn.com/apis/site/v2/sports/football/nfl/scoreboard?limit=1000&dates=2024&seasontype=2&week=7
https://site.api.espn.com/apis/site/v2/sports/football/nfl/scoreboard?limit=1000&dates=2024&seasontype=2&week=8
https://site.api.espn.com/apis/site/v2/sports/football/nfl/scoreboard?limit=1000&dates=2024&seasontype=2

100%|██████████| 166/166 [00:46<00:00,  3.53it/s]


In [3]:
from itertools import combinations

# Get all unique teams

drive_summary['result'] = drive_summary['result'].map({'W': 1, 'L': 0})
teams = drive_summary['team'].unique()

# Generate all possible team matchups
matchups = []
for team_a, team_b in combinations(teams, 2):  # Pairwise combinations of teams
    team_a_games = drive_summary[drive_summary['team'] == team_a]
    team_b_games = drive_summary[drive_summary['team'] == team_b]

    # Create pairwise matchups for all games between the two teams
    for _, team_a_game in team_a_games.iterrows():
        for _, team_b_game in team_b_games.iterrows():
            matchup = {
                **{f"Team_A_{col}": team_a_game[col] for col in drive_summary.columns if col not in ['team', 'opponent', 'date', 'result']},
                **{f"Team_B_{col}": team_b_game[col] for col in drive_summary.columns if col not in ['team', 'opponent', 'date', 'result']},
                "Result": 1 if team_a_game['result'] == 1 and team_b_game['result'] == 0 else 0
            }
            matchups.append(matchup)

# Convert to DataFrame
general_matchup_df = pd.DataFrame(matchups)


In [4]:
# Prepare the data
import joblib

X = general_matchup_df.drop(columns=['Result'])  # Features
y = general_matchup_df['Result']                # Target

# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
from sklearn.metrics import accuracy_score, classification_report
y_pred = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
print(classification_report(y_test, y_pred))

# Step 4: Save the trained model
joblib.dump(model, 'random_forest_matchup_model.pkl')
print("Model saved as 'random_forest_matchup_model.pkl'")


Accuracy: 1.00
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7838
           1       1.00      1.00      1.00      2840

    accuracy                           1.00     10678
   macro avg       1.00      1.00      1.00     10678
weighted avg       1.00      1.00      1.00     10678

Model saved as 'random_forest_matchup_model.pkl'


In [5]:
#### feature importance evaluation 

import pandas as pd

# Get feature importances
feature_importances = model.feature_importances_
feature_names = X.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

print("\nTop Feature Importances:")
print(importance_df.head(10))  # Top 10 features


Top Feature Importances:
                                Feature  Importance
25      Team_B_total_performance_metric    0.168865
12      Team_A_total_performance_metric    0.157280
24  Team_B_defensive_performance_metric    0.055635
2                        Team_A_point_%    0.053132
20            Team_B_point_prevention_%    0.053050
1                    Team_A_touchdown_%    0.048969
23  Team_B_offensive_performance_metric    0.046290
11  Team_A_defensive_performance_metric    0.044206
6         Team_A_touchdown_prevention_%    0.040381
10  Team_A_offensive_performance_metric    0.040214


In [7]:
#### cross validation ###

from sklearn.model_selection import cross_val_scoree

# Perform 5-fold cross-validation
cv_scores = cross_val_score(model, X, y, cv=5)
print(f"Cross-Validation Scores: {cv_scores}")
print(f"Mean CV Accuracy: {cv_scores.mean():.2f}")

Cross-Validation Scores: [0.94970968 0.91973401 0.95935188 0.96019481 0.95972651]
Mean CV Accuracy: 0.95


In [14]:
# Load the trained model
import joblib
model = joblib.load('random_forest_matchup_model.pkl')

# Function to create a matchup for prediction
def predict_matchup(team_a, team_b, drive_summary):

    numeric_columns = [col for col in drive_summary.select_dtypes(include=['float64', 'int64']).columns if col != 'result']
    team_a_stats = drive_summary[drive_summary['team'] == team_a][numeric_columns].mean()
    team_b_stats = drive_summary[drive_summary['team'] == team_b][numeric_columns].mean()

    matchup = {
        **{f"Team_A_{col}": team_a_stats[col] for col in numeric_columns},
        **{f"Team_B_{col}": team_b_stats[col] for col in numeric_columns}
    }

    matchup_df = pd.DataFrame([matchup])
    prediction = model.predict(matchup_df)
    print(prediction)
    return "Team A Wins" if prediction[0] == 1 else "Team B Wins"


matchup_1 = predict_matchup('Pittsburgh Steelers', 'Dallas Cowboys', drive_summary)
matchup_1


[1]


'Team A Wins'

In [3]:
drive_summary

,team,opponent,result,date,punt_%,touchdown_%,point_%,turnover_%,splash_allowed_%,forced_punt_%,touchdown_prevention_%,point_prevention_%,forced_turnover_%,splash_forced_%,offensive_performance_metric,defensive_performance_metric,total_performance_metric
0,Arizona Cardinals,Buffalo Bills,L,2024-09-08 17:00:00+00:00,0.200000,0.300000,0.500000,0.100000,0.100000,0.200000,0.600000,0.400000,0.100000,0.100000,0.500000,1.300000,1.800000
1,Arizona Cardinals,Los Angeles Rams,W,2024-09-15 20:05:00+00:00,0.111111,0.555556,0.777778,0.111111,0.111111,0.400000,0.900000,0.800000,0.100000,0.100000,1.111111,2.200000,3.311111
2,Arizona Cardinals,Detroit Lions,L,2024-09-22 20:25:00+00:00,0.454545,0.090909,0.272727,0.090909,0.090909,0.454545,0.727273,0.727273,0.090909,0.090909,-0.181818,2.000000,1.818182
3,Arizona Cardinals,Washington Commanders,L,2024-09-29 20:05:00+00:00,0.444444,0.222222,0.222222,0.111111,0.111111,0.111111,0.444444,0.222222,0.111111,0.111111,-0.111111,0.888889,0.777778
4,Arizona Cardinals,San Francisco 49ers,W,2024-10-06 20:05:00+00:00,0.200000,0.200000,0.500000,0.100000,0.200000,0.111111,0.888889,0.555556,0.333333,0.333333,0.300000,1.888889,2.188889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327,Washington Commanders,Carolina Panthers,W,2024-10-20 20:05:00+00:00,0.111111,0.333333,0.777778,0.000000,0.000000,0.500000,0.900000,0.900000,0.200000,0.200000,1.000000,2.500000,3.500000
328,Washington Commanders,Chicago Bears,W,2024-10-27 20:25:00+00:00,0.416667,0.083333,0.416667,0.000000,0.000000,0.636364,0.818182,0.818182,0.090909,0.090909,0.083333,2.363636,2.446970
329,Washington Commanders,New York Giants,W,2024-11-03 18:00:00+00:00,0.250000,0.375000,0.625000,0.000000,0.000000,0.250000,0.625000,0.500000,0.125000,0.125000,0.750000,1.500000,2.250000
330,Washington Commanders,Pittsburgh Steelers,L,2024-11-10 18:00:00+00:00,0.500000,0.250000,0.416667,0.000000,0.000000,0.357143,0.714286,0.714286,0.142857,0.142857,0.166667,1.928571,2.095238


In [4]:
def generate_matchups(drive_summary, team_a_name, team_b_name):
    """
    Generate game-by-game matchups between two specified teams.

    Args:
        drive_summary (pd.DataFrame): The full dataset containing game stats for all teams.
        team_a_name (str): Name of Team A (e.g., "Pittsburgh Steelers").
        team_b_name (str): Name of Team B (e.g., "Baltimore Ravens").

    Returns:
        pd.DataFrame: DataFrame of matchups with Team A and Team B stats, and a Result column.
    """
    # Filter the dataframe for the two specified teams
    drive_summary['result'] = drive_summary['result'].map({'W': 1, 'L': 0})
    team_subset = drive_summary[drive_summary['team'].isin([team_a_name, team_b_name])]
    
    # Select only numerical stats for simplicity
    numerical_stats = team_subset.drop(columns=['opponent', 'date', 'result', 'team']).select_dtypes(include=['float64', 'int64'])

    # Split the data by team
    team_a_stats = team_subset[team_subset['team'] == team_a_name]
    team_b_stats = team_subset[team_subset['team'] == team_b_name]

    matchups = []

    # Pair each game for Team A with each game for Team B
    for _, team_a_game in team_a_stats.iterrows():
        for _, team_b_game in team_b_stats.iterrows():
            matchup = {
                **{f"Team_A_{col}": team_a_game[col] for col in numerical_stats.columns},
                **{f"Team_B_{col}": team_b_game[col] for col in numerical_stats.columns},
                # Result logic: Adjust based on how you define matchups
                "Result": 1 if team_a_game['result'] == 1 and team_b_game['result'] == 0 else 0
            }
            matchups.append(matchup)

    # Convert matchups into a DataFrame
    matchup_df = pd.DataFrame(matchups)
    return matchup_df



In [5]:
matchup_df = generate_matchups(drive_summary, "Pittsburgh Steelers", "Baltimore Ravens")
matchup_df

,Team_A_punt_%,Team_A_touchdown_%,Team_A_point_%,Team_A_turnover_%,Team_A_splash_allowed_%,Team_A_forced_punt_%,Team_A_touchdown_prevention_%,Team_A_point_prevention_%,Team_A_forced_turnover_%,Team_A_splash_forced_%,...,Team_B_splash_allowed_%,Team_B_forced_punt_%,Team_B_touchdown_prevention_%,Team_B_point_prevention_%,Team_B_forced_turnover_%,Team_B_splash_forced_%,Team_B_offensive_performance_metric,Team_B_defensive_performance_metric,Team_B_total_performance_metric,Result
0,0.300000,0.0,0.600000,0.000000,0.000000,0.400000,0.900000,0.800000,0.300000,0.300000,...,0.100000,0.300000,0.700000,0.500000,0.100000,0.100000,0.300000,1.600000,1.900000,1
1,0.300000,0.0,0.600000,0.000000,0.000000,0.400000,0.900000,0.800000,0.300000,0.300000,...,0.083333,0.250000,0.833333,0.500000,0.083333,0.083333,0.166667,1.666667,1.833333,1
2,0.300000,0.0,0.600000,0.000000,0.000000,0.400000,0.900000,0.800000,0.300000,0.300000,...,0.000000,0.363636,0.727273,0.545455,0.090909,0.090909,0.363636,1.727273,2.090909,0
3,0.300000,0.0,0.600000,0.000000,0.000000,0.400000,0.900000,0.800000,0.300000,0.300000,...,0.090909,0.545455,0.909091,0.818182,0.090909,0.090909,0.454545,2.363636,2.818182,0
4,0.300000,0.0,0.600000,0.000000,0.000000,0.400000,0.900000,0.800000,0.300000,0.300000,...,0.153846,0.416667,0.583333,0.583333,0.083333,0.083333,0.538462,1.666667,2.205128,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,0.307692,0.0,0.461538,0.076923,0.076923,0.307692,0.846154,0.769231,0.230769,0.230769,...,0.100000,0.166667,0.666667,0.583333,0.166667,0.166667,0.900000,1.583333,2.483333,0
106,0.307692,0.0,0.461538,0.076923,0.076923,0.307692,0.846154,0.769231,0.230769,0.230769,...,0.000000,0.272727,0.727273,0.454545,0.090909,0.090909,0.272727,1.545455,1.818182,1
107,0.307692,0.0,0.461538,0.076923,0.076923,0.307692,0.846154,0.769231,0.230769,0.230769,...,0.000000,0.200000,0.900000,0.800000,0.100000,0.100000,1.000000,2.000000,3.000000,0
108,0.307692,0.0,0.461538,0.076923,0.076923,0.307692,0.846154,0.769231,0.230769,0.230769,...,0.000000,0.272727,0.545455,0.545455,0.090909,0.090909,0.416667,1.454545,1.871212,0


In [6]:
##### model ####


from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

# Step 1: Prepare the data
X = matchup_df.drop(columns=['Result']) 
y = matchup_df['Result']               

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 2: Train the Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Step 3: Evaluate the model
y_pred = model.predict(X_test)

# Print evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Step 4: Save the trained model
joblib.dump(model, 'random_forest_matchup_model.pkl')
print("Model saved as 'random_forest_matchup_model.pkl'")




Model Accuracy: 1.00

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       1.00      1.00      1.00         8

    accuracy                           1.00        22
   macro avg       1.00      1.00      1.00        22
weighted avg       1.00      1.00      1.00        22

Model saved as 'random_forest_matchup_model.pkl'


In [7]:
#### feature importance evaluation 

import pandas as pd

# Get feature importances
feature_importances = model.feature_importances_
feature_names = X.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

print("\nTop Feature Importances:")
print(importance_df.head(10))  # Top 10 features


Top Feature Importances:
                                Feature  Importance
14                   Team_B_touchdown_%    0.188212
23  Team_B_offensive_performance_metric    0.171678
20            Team_B_point_prevention_%    0.090767
12      Team_A_total_performance_metric    0.078005
25      Team_B_total_performance_metric    0.076490
15                       Team_B_point_%    0.058229
18                 Team_B_forced_punt_%    0.037356
7             Team_A_point_prevention_%    0.035747
6         Team_A_touchdown_prevention_%    0.032764
11  Team_A_defensive_performance_metric    0.031925


In [9]:
#### cross validation ###

from sklearn.model_selection import cross_val_score

# Perform 5-fold cross-validation
cv_scores = cross_val_score(model, X, y, cv=5)
print(f"Cross-Validation Scores: {cv_scores}")
print(f"Mean CV Accuracy: {cv_scores.mean():.2f}")

Cross-Validation Scores: [1.         1.         0.81818182 1.         1.        ]
Mean CV Accuracy: 0.96


In [53]:
# Load the trained model
import joblib
model = joblib.load('random_forest_matchup_model.pkl')

# Prepare the stats for the new teams
# Prepare the stats for the new teams
team_a_stats = drive_summary[drive_summary['team'] == 'Buffalo Bills'].select_dtypes(include=['float64', 'int64']).mean()
team_b_stats = drive_summary[drive_summary['team'] == 'Pittsburgh Steelers'].select_dtypes(include=['float64', 'int64']).mean()

# Create the new matchup dictionary
new_matchup = {
    **{f"Team_A_{col}": team_a_stats[col] for col in drive_summary.columns if col not in ['team', 'opponent', 'date', 'result']},
    **{f"Team_B_{col}": team_b_stats[col] for col in drive_summary.columns if col not in ['team', 'opponent', 'date', 'result']}
}

# Convert to DataFrame
new_matchup_df = pd.DataFrame([new_matchup])

# Predict the outcome
prediction = model.predict(new_matchup_df)
print("Predicted Result:", "Team A Wins" if prediction[0] == 1 else "Team B Wins")



Predicted Result: Team B Wins


In [32]:
new_matchup_df

,Team_A_punt_%,Team_A_touchdown_%,Team_A_point_%,Team_A_turnover_%,Team_A_splash_allowed_%,Team_A_forced_punt_%,Team_A_touchdown_prevention_%,Team_A_point_prevention_%,Team_A_forced_turnover_%,Team_A_splash_forced_%,...,Team_B_turnover_%,Team_B_splash_allowed_%,Team_B_forced_punt_%,Team_B_touchdown_prevention_%,Team_B_point_prevention_%,Team_B_forced_turnover_%,Team_B_splash_forced_%,Team_B_offensive_performance_metric,Team_B_defensive_performance_metric,Team_B_total_performance_metric
0,0.3,0.0,0.6,0.0,0.0,0.4,0.9,0.8,0.3,0.3,...,0.0,0.0,0.333333,0.666667,0.666667,0.0,0.0,1.111111,1.666667,2.777778


# Attempt Two 

Accuracy: 1.00
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7838
           1       1.00      1.00      1.00      2840

    accuracy                           1.00     10678
   macro avg       1.00      1.00      1.00     10678
weighted avg       1.00      1.00      1.00     10678

Model saved as 'random_forest_matchup_model.pkl'


In [10]:

import pandas as pd

# Get feature importances
feature_importances = model.feature_importances_
feature_names = X.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

print("\nTop Feature Importances:")
print(importance_df.head(10))  # Top 10 features


Top Feature Importances:
                                Feature  Importance
25      Team_B_total_performance_metric    0.168865
12      Team_A_total_performance_metric    0.157280
24  Team_B_defensive_performance_metric    0.055635
2                        Team_A_point_%    0.053132
20            Team_B_point_prevention_%    0.053050
1                    Team_A_touchdown_%    0.048969
23  Team_B_offensive_performance_metric    0.046290
11  Team_A_defensive_performance_metric    0.044206
6         Team_A_touchdown_prevention_%    0.040381
10  Team_A_offensive_performance_metric    0.040214


In [11]:
#### cross validation ###

from sklearn.model_selection import cross_val_score

# Perform 5-fold cross-validation
cv_scores = cross_val_score(model, X, y, cv=5)
print(f"Cross-Validation Scores: {cv_scores}")
print(f"Mean CV Accuracy: {cv_scores.mean():.2f}")

Cross-Validation Scores: [0.94970968 0.91973401 0.95935188 0.96019481 0.95972651]
Mean CV Accuracy: 0.95


In [137]:
# Load the trained model
import joblib
model = joblib.load('random_forest_matchup_model.pkl')

# Function to create a matchup for prediction
def predict_matchup(team_a, team_b, drive_summary):
    # Ensure only numeric stats are used, explicitly excluding the 'result' column
    numeric_columns = [col for col in drive_summary.select_dtypes(include=['float64', 'int64']).columns if col != 'result']

    # Get stats for both teams
    team_a_stats = drive_summary[drive_summary['team'] == team_a][numeric_columns].mean()
    team_b_stats = drive_summary[drive_summary['team'] == team_b][numeric_columns].mean()

    # Create the matchup dictionary
    matchup = {
        **{f"Team_A_{col}": team_a_stats[col] for col in numeric_columns},
        **{f"Team_B_{col}": team_b_stats[col] for col in numeric_columns}
    }

    # Convert to DataFrame
    matchup_df = pd.DataFrame([matchup])

    # Predict the outcome
    prediction = model.predict(matchup_df)
    print(prediction)
    return "Team A Wins" if prediction[0] == 1 else "Team B Wins"

# Example: Predict a matchup
#teams = drive_summary['team'].unique()
matchup_1 = predict_matchup('Dallas Cowboys', 'Houston Texans', drive_summary)
matchup_1


[0]


'Team B Wins'

array([0])